In [3]:
#Default
import pandas as pd
pd.options.display.float_format = '{:,}'.format
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
def reset(df):
    cols = df.columns
    return df.reset_index()[cols]
def print_counts(df):
    cols = df.columns
    for each in cols:
        print(each)
        print(df[each].value_counts())
        print('______________________________________')
# ~
#Default Ending

import os
from tqdm import tqdm

print('OK !')

OK !


In [4]:
from haystack.utils import clean_wiki_text, convert_files_to_dicts, fetch_archive_from_http, print_answers
from haystack.nodes import DensePassageRetriever, EmbeddingRetriever, FARMReader, EntityExtractor
from haystack.document_stores import ElasticsearchDocumentStore

print('OK !')

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/


OK !


# Set similarity_type

In [5]:
# For DensePassageRetriever
similarity_type = "dot_product"

# For EmbeddingRetriever
# similarity_type = "cosine"

In [6]:
document_store_100 = ElasticsearchDocumentStore(host="localhost", port = "9200", index="document_100",
                                           similarity=similarity_type)
document_store_200 = ElasticsearchDocumentStore(host="localhost", port = "9200", index="document_200",
                                           similarity=similarity_type)
document_store_300 = ElasticsearchDocumentStore(host="localhost", port = "9200", index="document_300",
                                           similarity=similarity_type)

In [8]:
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor

preprocessor_100 = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
)

preprocessor_200 = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=200,
    split_respect_sentence_boundary=True,
)

preprocessor_300 = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=300,
    split_respect_sentence_boundary=True,
)

converter = PDFToTextConverter(remove_numeric_tables=False, valid_languages = ["en"])

print('OK !')

OK !


pdftotext version 4.03 [www.xpdfreader.com]
Copyright 1996-2021 Glyph & Cog, LLC


# Set document_store & preprocessor 

In [9]:
document_store = document_store_300
preprocessor = preprocessor_300

# Process Raw PDF

In [10]:
def clean_content(content):
    content = content.replace('..','.')
    content = content.replace('..','.')
    content = content.replace('..','.')
    content = content.replace('..','.')
    content = content.replace('..','.')
    content = content.replace('..','.')
    content = content.replace('..','.')
    content = content.replace('..','.')
    content = content.replace('..','.')
    content = content.replace('..','.')
    return content

entity_extractor = EntityExtractor(model_name_or_path="dslim/bert-base-NER")

print('OK !')

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0


OK !


In [11]:
root = 'International Regulator'

file_list = []
for path, subdirs, files in os.walk(root):
    for name in files:
#         print(os.path.join(path, name))
        file_list.append(os.path.join(path, name))
    
len(file_list)

88

# Clean Text & NER Extraction

In [12]:
doc_list = []
for file_name in tqdm(file_list):
    doc = converter.convert(file_path = file_name, meta = None)
    doc[0]['content'] = clean_content(doc[0]['content'])
    docs = preprocessor.process(doc[0])
    
    keyword = file_name.split('/')[-2]
    central_bank = file_name.split('/')[-3]
    
    for doc in docs:
        doc['meta']['keyword'] = keyword
        doc['meta']['central_bank'] = central_bank
        doc['meta']['file_name'] = file_name
        
        content = doc['content']
        entities = entity_extractor.extract(
            text=content
         )
        
        e_df = pd.DataFrame(entities)
        if len(e_df) > 0:
            e_df = e_df.groupby('word').first().reset_index()
            e_df = e_df.groupby('entity_group')['word'].apply(list).reset_index()
            for j in range(len(e_df)):
                doc['meta'][e_df['entity_group'].values[j]] = e_df['word'].values[j]
            
        doc_list = doc_list + [doc]
#         break 
#     
#     break

  1%|▍                                        | 1/88 [03:06<4:31:06, 186.97s/it]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  2%|▉                                        | 2/88 [04:49<3:16:43, 137.26s/it]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  3%|█▍                                        | 3/88 [05:07<1:57:03, 82.63s/it]

...

 98%|████████████████████████████████████████ | 86/88 [3:00:49<02:09, 64.86s/it]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 99%|████████████████████████████████████████▌| 87/88 [3:01:47<01:03, 63.33s/it]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|████████████████████████████████████████| 88/88 [3:01:51<00:00, 124.00s/it]


In [13]:
document_store.delete_documents()

In [14]:
document_store.write_documents(doc_list)

In [15]:
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="DPR-retriever/query_encoder",
    passage_embedding_model="DPR-retriever/passage_encoder",
    max_seq_len_query=64,
    max_seq_len_passage=256,
    batch_size=16,
    use_gpu=True,
    embed_title=True,
    use_fast_tokenizers=True,
)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at DPR-retriever/query_encoder
INFO - haystack.modeling.model.language_model -  Loaded DPR-retriever/query_encoder
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at DPR-retriever/passage_encoder
INFO - haystack.modeling.model.language_model -  Loaded DPR-retriever/passage_encoder


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
document_store.update_embeddings(retriever)

INFO - haystack.document_stores.elasticsearch -  Updating embeddings for all 4551 docs ...


Updating embeddings:   0%|          | 0/4551 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/4560 [00:00<?, ? Docs/s]